In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, Activation, Dropout, Flatten, Dense
from keras.layers import GlobalMaxPooling2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

%matplotlib inline

Using TensorFlow backend.


In [2]:
DATA_FILE = '../data/train_data.h5'
METADATA = '../metadata.npy'

f = h5py.File(DATA_FILE, 'r')
data_x = f['x']
data_y = f['y']
metadata = np.load(METADATA).item()

In [3]:
data_x = np.asarray(data_x)
data_y = np.asarray(data_y)

# Metrics

In [4]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    recall = recall(y_true, y_pred)
    precision = precision(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

# Stratified CNN

In [5]:
def create_model():
    lr = 1e-3
    n_conv_layers = 3
    n_kernels = [128] * n_conv_layers
    mp_sizes = [(2,2) for _ in range(n_conv_layers)]
    dropout_values = [0.25] * n_conv_layers

    model = Sequential()
    model.add(BatchNormalization(axis=3, input_shape=data_x.shape[1:]))
    for i in range(n_conv_layers):
        model.add(Conv2D(n_kernels[i], (3, 3), padding="same", kernel_initializer="he_uniform"))
        model.add(BatchNormalization(axis=3))
        model.add(Activation('relu'))
        if i < n_conv_layers - 1:
            model.add(Dropout(dropout_values[i]))

    model.add(Conv2D(12, (3, 3), padding="same"))
    model.add(GlobalMaxPooling2D())
    model.add(Activation('softmax'))
    model.compile(Adam(lr), loss='categorical_crossentropy', metrics=['accuracy', recall, precision, f1])
    return model

In [6]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

cnn_models = []
kf = StratifiedKFold(n_splits=7)
kf_index = 1
for train, val in kf.split(data_x, data_y.argmax(1)):
    model = create_model()
    model_filepath = '../models/Kaggle/CNN/KF_CNN_Model' + str(kf_index) + '.h5'
    callbacks = [ModelCheckpoint(model_filepath, monitor='val_f1', verbose=1, save_best_only=True, mode="max")]
    X_train, X_val, y_train, y_val = data_x[train], data_x[val], data_y[train], data_y[val]
    model.fit(X_train, y_train, batch_size=64, epochs=25,
          validation_data=(X_val, y_val), callbacks=callbacks)
    cnn_models.append(model)
    kf_index += 1

Train on 4068 samples, validate on 682 samples
Epoch 1/25
4068/4068 [==============================] - 18s - loss: 1.4676 - acc: 0.5774 - recall: 0.2335 - precision: 0.7060 - f1: nan - val_loss: 1.7670 - val_acc: 0.7258 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_f1: nan
Epoch 2/25
4068/4068 [==============================] - 19s - loss: 0.5765 - acc: 0.8439 - recall: 0.7112 - precision: 0.9160 - f1: 0.7986 - val_loss: 0.8796 - val_acc: 0.8460 - val_recall: 0.4956 - val_precision: 0.9474 - val_f1: 0.6455
Epoch 3/25
4068/4068 [==============================] - 20s - loss: 0.3767 - acc: 0.8830 - recall: 0.8343 - precision: 0.9190 - f1: 0.8743 - val_loss: 0.4850 - val_acc: 0.8783 - val_recall: 0.7991 - val_precision: 0.9112 - val_f1: 0.8498
Epoch 4/25
4068/4068 [==============================] - 16s - loss: 0.2902 - acc: 0.9083 - recall: 0.8786 - precision: 0.9340 - f1: 0.9052 - val_loss: 0.4063 - val_acc: 0.8710 - val_recall: 0.8446 - val_precision: 0.8964 - val_f1: 0.8695

4068/4068 [==============================] - 17s - loss: 0.0586 - acc: 0.9803 - recall: 0.9786 - precision: 0.9820 - f1: 0.9803 - val_loss: 0.4800 - val_acc: 0.8856 - val_recall: 0.8827 - val_precision: 0.8931 - val_f1: 0.8878
Epoch 20/25
4068/4068 [==============================] - 17s - loss: 0.0623 - acc: 0.9784 - recall: 0.9764 - precision: 0.9807 - f1: 0.9785 - val_loss: 0.4348 - val_acc: 0.8856 - val_recall: 0.8827 - val_precision: 0.8914 - val_f1: 0.8870
Epoch 21/25
4068/4068 [==============================] - 16s - loss: 0.0684 - acc: 0.9754 - recall: 0.9739 - precision: 0.9785 - f1: 0.9762 - val_loss: 0.5012 - val_acc: 0.8856 - val_recall: 0.8798 - val_precision: 0.8905 - val_f1: 0.8851
Epoch 22/25
4068/4068 [==============================] - 16s - loss: 0.0597 - acc: 0.9835 - recall: 0.9818 - precision: 0.9849 - f1: 0.9834 - val_loss: 0.4613 - val_acc: 0.8974 - val_recall: 0.8915 - val_precision: 0.9050 - val_f1: 0.8981
Epoch 23/25
4068/4068 [==============================] -

4068/4068 [==============================] - 16s - loss: 0.0963 - acc: 0.9666 - recall: 0.9621 - precision: 0.9719 - f1: 0.9670 - val_loss: 0.3064 - val_acc: 0.9150 - val_recall: 0.9135 - val_precision: 0.9225 - val_f1: 0.9180
Epoch 14/25
4068/4068 [==============================] - 16s - loss: 0.0668 - acc: 0.9794 - recall: 0.9759 - precision: 0.9802 - f1: 0.9780 - val_loss: 0.2589 - val_acc: 0.9223 - val_recall: 0.9179 - val_precision: 0.9287 - val_f1: 0.9232
Epoch 15/25
4068/4068 [==============================] - 16s - loss: 0.0654 - acc: 0.9798 - recall: 0.9776 - precision: 0.9815 - f1: 0.9795 - val_loss: 0.2936 - val_acc: 0.9164 - val_recall: 0.9150 - val_precision: 0.9173 - val_f1: 0.9161
Epoch 16/25
4068/4068 [==============================] - 17s - loss: 0.0879 - acc: 0.9698 - recall: 0.9680 - precision: 0.9716 - f1: 0.9698 - val_loss: 0.2415 - val_acc: 0.9282 - val_recall: 0.9223 - val_precision: 0.9330 - val_f1: 0.9276
Epoch 17/25
4068/4068 [==============================] -

4069/4069 [==============================] - 17s - loss: 0.1956 - acc: 0.9356 - recall: 0.9191 - precision: 0.9509 - f1: 0.9347 - val_loss: 0.2602 - val_acc: 0.9016 - val_recall: 0.8899 - val_precision: 0.9148 - val_f1: 0.9020
Epoch 7/25
4069/4069 [==============================] - 17s - loss: 0.1711 - acc: 0.9425 - recall: 0.9302 - precision: 0.9531 - f1: 0.9414 - val_loss: 0.2509 - val_acc: 0.9060 - val_recall: 0.9001 - val_precision: 0.9166 - val_f1: 0.9082
Epoch 8/25
4069/4069 [==============================] - 17s - loss: 0.1557 - acc: 0.9504 - recall: 0.9393 - precision: 0.9557 - f1: 0.9474 - val_loss: 0.2481 - val_acc: 0.9134 - val_recall: 0.9075 - val_precision: 0.9272 - val_f1: 0.9171
Epoch 9/25
4069/4069 [==============================] - 17s - loss: 0.1390 - acc: 0.9528 - recall: 0.9459 - precision: 0.9593 - f1: 0.9525 - val_loss: 0.3159 - val_acc: 0.9060 - val_recall: 0.9016 - val_precision: 0.9108 - val_f1: 0.9062
Epoch 10/25
4069/4069 [==============================] - 17

4070/4070 [==============================] - 18s - loss: 1.4468 - acc: 0.5894 - recall: 0.2356 - precision: 0.7460 - f1: nan - val_loss: 1.7185 - val_acc: 0.7515 - val_recall: 0.0015 - val_precision: 0.0941 - val_f1: nan
Epoch 2/25
4070/4070 [==============================] - 17s - loss: 0.5912 - acc: 0.8344 - recall: 0.7076 - precision: 0.9154 - f1: 0.7961 - val_loss: 0.8504 - val_acc: 0.8529 - val_recall: 0.5221 - val_precision: 0.9561 - val_f1: 0.6640
Epoch 3/25
4070/4070 [==============================] - 17s - loss: 0.3806 - acc: 0.8833 - recall: 0.8339 - precision: 0.9198 - f1: 0.8744 - val_loss: 0.4714 - val_acc: 0.8603 - val_recall: 0.8029 - val_precision: 0.9308 - val_f1: 0.8609
Epoch 4/25
4070/4070 [==============================] - 17s - loss: 0.2929 - acc: 0.9052 - recall: 0.8717 - precision: 0.9343 - f1: 0.9017 - val_loss: 0.3673 - val_acc: 0.8750 - val_recall: 0.8382 - val_precision: 0.9109 - val_f1: 0.8727
Epoch 5/25
4070/4070 [==============================] - 17s - los

4070/4070 [==============================] - 17s - loss: 0.0471 - acc: 0.9840 - recall: 0.9828 - precision: 0.9859 - f1: 0.9844 - val_loss: 0.4672 - val_acc: 0.8912 - val_recall: 0.8882 - val_precision: 0.8986 - val_f1: 0.8933
Epoch 20/25
4070/4070 [==============================] - 16s - loss: 0.0507 - acc: 0.9840 - recall: 0.9823 - precision: 0.9847 - f1: 0.9835 - val_loss: 0.5209 - val_acc: 0.8794 - val_recall: 0.8735 - val_precision: 0.8848 - val_f1: 0.8791
Epoch 21/25
4070/4070 [==============================] - 16s - loss: 0.0550 - acc: 0.9799 - recall: 0.9784 - precision: 0.9829 - f1: 0.9806 - val_loss: 0.5522 - val_acc: 0.8765 - val_recall: 0.8721 - val_precision: 0.8768 - val_f1: 0.8744
Epoch 22/25
4070/4070 [==============================] - 16s - loss: 0.0731 - acc: 0.9757 - recall: 0.9740 - precision: 0.9771 - f1: 0.9755 - val_loss: 0.4948 - val_acc: 0.8794 - val_recall: 0.8779 - val_precision: 0.8857 - val_f1: 0.8817
Epoch 23/25
4070/4070 [==============================] -

4073/4073 [==============================] - 17s - loss: 0.0836 - acc: 0.9725 - recall: 0.9681 - precision: 0.9746 - f1: 0.9713 - val_loss: 0.3608 - val_acc: 0.9069 - val_recall: 0.8996 - val_precision: 0.9184 - val_f1: 0.9088
Epoch 14/25
4073/4073 [==============================] - 17s - loss: 0.0814 - acc: 0.9715 - recall: 0.9688 - precision: 0.9750 - f1: 0.9719 - val_loss: 0.3657 - val_acc: 0.8877 - val_recall: 0.8848 - val_precision: 0.9008 - val_f1: 0.8927
Epoch 15/25
4073/4073 [==============================] - 17s - loss: 0.0689 - acc: 0.9767 - recall: 0.9752 - precision: 0.9798 - f1: 0.9775 - val_loss: 0.3545 - val_acc: 0.8996 - val_recall: 0.8951 - val_precision: 0.9058 - val_f1: 0.9004
Epoch 16/25
4073/4073 [==============================] - 17s - loss: 0.0651 - acc: 0.9808 - recall: 0.9769 - precision: 0.9834 - f1: 0.9801 - val_loss: 0.3384 - val_acc: 0.9069 - val_recall: 0.9040 - val_precision: 0.9174 - val_f1: 0.9106
Epoch 17/25
4073/4073 [==============================] -

4075/4075 [==============================] - 18s - loss: 0.1659 - acc: 0.9467 - recall: 0.9389 - precision: 0.9546 - f1: 0.9466 - val_loss: 0.2619 - val_acc: 0.9096 - val_recall: 0.8978 - val_precision: 0.9237 - val_f1: 0.9104
Epoch 8/25
4075/4075 [==============================] - 17s - loss: 0.1450 - acc: 0.9512 - recall: 0.9433 - precision: 0.9603 - f1: 0.9517 - val_loss: 0.2660 - val_acc: 0.9096 - val_recall: 0.9037 - val_precision: 0.9207 - val_f1: 0.9119
Epoch 9/25
4075/4075 [==============================] - 17s - loss: 0.1491 - acc: 0.9477 - recall: 0.9394 - precision: 0.9568 - f1: 0.9480 - val_loss: 0.2549 - val_acc: 0.9200 - val_recall: 0.9156 - val_precision: 0.9281 - val_f1: 0.9217
Epoch 10/25
4075/4075 [==============================] - 17s - loss: 0.1195 - acc: 0.9602 - recall: 0.9544 - precision: 0.9643 - f1: 0.9592 - val_loss: 0.2809 - val_acc: 0.9156 - val_recall: 0.9052 - val_precision: 0.9217 - val_f1: 0.9132
Epoch 11/25
4075/4075 [==============================] - 1

4077/4077 [==============================] - 17s - loss: 0.5931 - acc: 0.8347 - recall: 0.7062 - precision: 0.9107 - f1: 0.7938 - val_loss: 0.8317 - val_acc: 0.8811 - val_recall: 0.5126 - val_precision: 0.9806 - val_f1: 0.6537
Epoch 3/25
4077/4077 [==============================] - 17s - loss: 0.3710 - acc: 0.8855 - recall: 0.8339 - precision: 0.9221 - f1: 0.8754 - val_loss: 0.4367 - val_acc: 0.8945 - val_recall: 0.7920 - val_precision: 0.9499 - val_f1: 0.8615
Epoch 4/25
4077/4077 [==============================] - 17s - loss: 0.2807 - acc: 0.9142 - recall: 0.8788 - precision: 0.9349 - f1: 0.9058 - val_loss: 0.3367 - val_acc: 0.8826 - val_recall: 0.8425 - val_precision: 0.9129 - val_f1: 0.8757
Epoch 5/25
4077/4077 [==============================] - 17s - loss: 0.2321 - acc: 0.9237 - recall: 0.9029 - precision: 0.9410 - f1: 0.9214 - val_loss: 0.2853 - val_acc: 0.9004 - val_recall: 0.8856 - val_precision: 0.9176 - val_f1: 0.9011
Epoch 6/25
4077/4077 [==============================] - 17s

4077/4077 [==============================] - 17s - loss: 0.0654 - acc: 0.9784 - recall: 0.9767 - precision: 0.9810 - f1: 0.9789 - val_loss: 0.3459 - val_acc: 0.9049 - val_recall: 0.9034 - val_precision: 0.9119 - val_f1: 0.9076
Epoch 21/25
4077/4077 [==============================] - 17s - loss: 0.0597 - acc: 0.9818 - recall: 0.9806 - precision: 0.9838 - f1: 0.9822 - val_loss: 0.4343 - val_acc: 0.9034 - val_recall: 0.9004 - val_precision: 0.9044 - val_f1: 0.9024
Epoch 22/25
4077/4077 [==============================] - 17s - loss: 0.0546 - acc: 0.9845 - recall: 0.9833 - precision: 0.9852 - f1: 0.9843 - val_loss: 0.3683 - val_acc: 0.8990 - val_recall: 0.8945 - val_precision: 0.9013 - val_f1: 0.8979
Epoch 23/25
4077/4077 [==============================] - 16s - loss: 0.0494 - acc: 0.9823 - recall: 0.9809 - precision: 0.9840 - f1: 0.9824 - val_loss: 0.3895 - val_acc: 0.8915 - val_recall: 0.8886 - val_precision: 0.8954 - val_f1: 0.8919
Epoch 24/25
4077/4077 [==============================] -